In [3]:
import findspark
findspark.init()
import pyspark
from pyspark.sql import SparkSession, SQLContext
from pyspark.sql.functions import col
#from elan.qelanmodel import QElanModel
from pyspark.sql.types import *
from pyspark.sql.functions import pandas_udf, PandasUDFType
from pathlib import Path
import os 
import pandas as pa
    
spark = SparkSession\
        .builder\
        .appName("PythonPi")\
        .getOrCreate()

sc = spark.sparkContext
sqlContext = SQLContext(sc)


In [ ]:
df = spark.read.parquet("/user/Current_Data/partitioned_key=MEDIA/"

In [4]:
csv_df = pa.read_csv("/media/sf_share/EQUINOR/OGA_Wells_ED50.csv")
# csv_df.describe()

,X,Y,OBJECTID,DENNO,WELLSYMBOL,TOPHOLEYDD,TOPHOLEXDD,TOPHOLEYNG,TOPHOLEXNG,QUADRANTNO,...,WATDEP_F,RELEASENO,TDTVDSSM,TDTVDSSF,TDMDDEPM,TDMDDEPF,TDDATDEP_M,TDDATDEP_F,LICCONFPER,LICNO
count,12105.000000,12105.000000,12184.000000,11722.000000,11741.000000,12108.000000,12108.000000,1.0,1.0,12113.000000,...,12107.000000,2978.000000,10896.000000,10896.000000,11151.000000,11151.000000,11230.000000,11230.000000,1.0,12142.000000
mean,0.838969,57.804235,6092.500000,6935.692032,12.935781,57.804043,0.839244,500756.0,503210.0,57.928176,...,385.009316,20.295836,2589.469369,8495.634591,3469.440364,11382.678572,36.875055,120.981151,5.0,309.116208
std,1.639153,2.631243,3517.362175,6332.395058,6.786797,2.630946,1.639043,NaN,NaN,71.863403,...,362.240596,13.018402,1169.628748,3837.364749,1193.355716,3915.209121,14.567943,47.795089,NaN,346.208600
min,-9.843642,48.525472,1.000000,1.000000,1.000000,48.525472,-9.843643,500756.0,503210.0,1.000000,...,0.000000,1.000000,-2641.396800,-8666.000000,0.000000,0.000000,0.000000,0.000000,5.0,1.000000
25%,0.720056,56.662206,3046.750000,2633.250000,7.000000,56.671009,0.720571,500756.0,503210.0,15.000000,...,264.000000,8.000000,1996.135200,6549.000000,2718.407950,8918.661550,25.908000,85.000000,5.0,104.000000
50%,1.302861,58.090919,6092.500000,5177.500000,14.000000,58.089538,1.302951,500756.0,503210.0,22.000000,...,364.004400,19.000000,2717.292000,8915.000150,3450.950000,11322.014800,32.918400,108.000000,5.0,218.000000
75%,1.714526,59.758439,9138.250000,7932.750000,19.000000,59.757214,1.714705,500756.0,503210.0,49.000000,...,460.000000,33.000000,3329.025600,10922.000000,4243.083800,13920.879050,46.939200,154.000000,5.0,324.000000
max,3.354361,62.656761,12184.000000,25090.000000,38.000000,62.656761,3.354361,500756.0,503210.0,220.000000,...,12034.121100,42.000000,8920.604000,29267.074400,9153.543000,30031.310000,198.424800,651.000000,5.0,2371.000000


In [10]:

def import_df(csv_df,sourcename,wbi_c,uid_c,property_type):

    csv_files = []
    csv_files.append(csv_df)

    def convert_to_parquet(csv_df,sourcename):
        dictarray = []  
        def to_dict_format(x):      
            descr = "from {}".format(sourcename)
            for c in x.index:
                unit = None
                if c == "Depth_MD":              
                    unit = "M"
                if c != wbi_c and c != uid_c :
                    mydict = {"properties":property_type,"wbi":x[wbi_c],"uid":x[uid_c],"source":sourcename,"mnemonic":c,"unit":unit,"value":x[c],"descr":descr }
                    dictarray.append(mydict)

        csv_df.apply(to_dict_format,axis=1)
        return dictarray


    schema = StructType([
        StructField("properties", StringType(), True),    
        StructField("source", StringType(), True), 
        StructField("wbi", StringType(), True), 
        StructField("uid", StringType(), True),   
        StructField("mnemonic", StringType(), True), 
        StructField("unit", StringType(), True), 
        StructField("value", StringType(), True),
        StructField("descr", StringType(), True)
    ])

    header = spark.sparkContext.parallelize(csv_files).flatMap(lambda x : convert_to_parquet(x,sourcename)).toDF(schema=schema)

    destfolder = "/home/molood/equinormetadata/origin={0}".format(sourcename)

    header.write.mode('overwrite').format("parquet").save(destfolder)



In [11]:
sourcename = "OGA_WELLS_ED50"
wbi_c = "WELLREGNO"
uid_c = wbi_c
property_type = "well"

import_df(csv_df,sourcename,wbi_c,uid_c,property_type)

In [18]:
df.to_parquet("/media/sf_share/equinor_metadata.parquet")

In [15]:
df_oga_50 = df[df.origin=="OGA_WELLS_ED50"]
df.wbi.nunique()

24659

In [3]:
import pandas as pa
df  = pa.read_parquet("/home/molood/equinormetadata")

In [4]:
df

,properties,source,wbi,mnemonic,unit,value,descr,origin
0,well,WellHdr_EquinorCDAWellHdrs,1/4/2001,WELLBORE_NAME,None,1/4/2001,from WellHdr_EquinorCDAWellHdrs,WellHdr_EquinorCDAWellHdrs
1,well,WellHdr_EquinorCDAWellHdrs,1/4/2001,WELL_ORIGIN,None,1/4/2001,from WellHdr_EquinorCDAWellHdrs,WellHdr_EquinorCDAWellHdrs
2,well,WellHdr_EquinorCDAWellHdrs,1/4/2001,SPUD_DATE,None,14-Oct-82,from WellHdr_EquinorCDAWellHdrs,WellHdr_EquinorCDAWellHdrs
3,well,WellHdr_EquinorCDAWellHdrs,1/4/2001,COMPLETION_DATE,None,25-Nov-82,from WellHdr_EquinorCDAWellHdrs,WellHdr_EquinorCDAWellHdrs
4,well,WellHdr_EquinorCDAWellHdrs,1/4/2001,WELL_INTENT,None,EXPLORATION,from WellHdr_EquinorCDAWellHdrs,WellHdr_EquinorCDAWellHdrs
5,well,WellHdr_EquinorCDAWellHdrs,1/4/2001,CURRENT_STATUS,None,P & A,from WellHdr_EquinorCDAWellHdrs,WellHdr_EquinorCDAWellHdrs
6,well,WellHdr_EquinorCDAWellHdrs,1/4/2001,CURRENT_OWNER,None,BP,from WellHdr_EquinorCDAWellHdrs,WellHdr_EquinorCDAWellHdrs
7,well,WellHdr_EquinorCDAWellHdrs,1/4/2001,FIELD_NAME,None,NaN,from WellHdr_EquinorCDAWellHdrs,WellHdr_EquinorCDAWellHdrs
8,well,WellHdr_EquinorCDAWellHdrs,1/4/2001,ORIGINAL_OPERATOR,None,BP EXPLORATION OPERATING COMPANY LIMITED,from WellHdr_EquinorCDAWellHdrs,WellHdr_EquinorCDAWellHdrs
9,well,WellHdr_EquinorCDAWellHdrs,1/4/2001,RELEASE_DATE,None,7-Jul-88,from WellHdr_EquinorCDAWellHdrs,WellHdr_EquinorCDAWellHdrs
